# Fluid dynamic computations for MRI simulation in Koma MRI
### Joaquín Anatol Hernández, PhD and José Sierra-Pallares, PhD
### Energy and Fluid Mechanics Department - UVa

In [1]:
import pyvista as pv
import vtk
import os
import numpy as np
import glob

## Auxiliary functions

In [2]:
def create_vti_file(filename, edge_size):
  mesh = pv.read(filename)
  points  = mesh.points

  root, ext = os.path.splitext(filename)

  image_box = [(np.max(points[:,0])-np.min(points[:,0])),\
          (np.max(points[:,1])-np.min(points[:,1])),\
          (np.max(points[:,2])-np.min(points[:,2]))]


  orig_image =[np.min(points,axis=0)[0], np.min(points,axis=0)[1], np.min(points,axis=0)[2]]

  image_size_x=edge_size
  image_size_y=edge_size
  image_size_z=edge_size

  nx = int(image_box[0]/image_size_x)
  ny = int(image_box[1]/image_size_y)
  nz = int(image_box[2]/image_size_z)

  res_x = image_box[0]/nx
  res_y = image_box[1]/ny
  res_z = image_box[2]/nz

  dims=(nx, ny, nz)
  spa=(res_x,res_y,res_z)
  orig=(orig_image[0],orig_image[1],orig_image[2])
  image = pv.ImageData(dimensions=dims,spacing=spa,origin=orig)

  #mesh=mesh.cell_data_to_point_data()
  probed_image=image.sample(mesh, mark_blank=True)
  probed_image = probed_image.point_data_to_cell_data()

  filename = root + '.vti'
  
  writer = vtk.vtkDataSetWriter()
  writer.SetFileTypeToASCII()
  writer.SetInputData(probed_image)
  # Set the output file name with .vtk extension
  writer.SetFileName(filename)
  writer.SetFileVersion(42)
  writer.SetWriteArrayMetaData(False)
  writer.Write()

  
def create_seed_file(vtifile, seedfile):
  probed_image = pv.read(vtifile)
  centers = probed_image.cell_centers()
  with open(seedfile, 'w') as file:
    for i, point in enumerate(centers.points):
      try:
        if centers['vtkGhostType'][i] <1:
          file.write(f'{point[0]} {point[1]} {point[2]}\n')
      except:
        file.write(f'{point[0]} {point[1]} {point[2]}\n')

def create_sUbend_files(step_size, end_time, list_of_files, filename='suBend.vti'):
  mesh = pv.read(filename)
  RST = mesh['RST']
  U = mesh['U']
  time_vector = np.linspace(0, end_time, int(end_time/step_size))
  file = open(list_of_files, 'w')

  for j, time in enumerate(time_vector):
    name = 'sUbend_'+str(j)+'.vtk'
    matrix_M = np.empty((3, 3))
    vt = np.empty_like(mesh['U'])
    for i, chol_ in enumerate(RST):
      matrix_M[0, 0] = chol_[0]
      matrix_M[0, 1] = chol_[3]
      matrix_M[0, 2] = chol_[5]
      matrix_M[1, 0] = chol_[3]
      matrix_M[1, 1] = chol_[1]
      matrix_M[1, 2] = chol_[4]
      matrix_M[2, 0] = chol_[5]
      matrix_M[2, 1] = chol_[3]
      matrix_M[2, 2] = chol_[2]

      random_vector = np.random.rand(3)

      vt[i] = np.dot(matrix_M, random_vector)*np.sqrt(1-np.exp(-2*step_size))
      U[i] = U[i] + vt[i]
      if i>1:
        U[i] = U[i] + vt[i-1]

    mesh.clear_data()
    mesh['U'] = U
    
    # Save the VTK legacy file
    writer = vtk.vtkDataSetWriter()
    writer.SetFileTypeToASCII()
    writer.SetInputData(mesh)
    # Set the output file name with .vtk extension
    writer.SetFileName(name)
    writer.SetFileVersion(42)
    writer.SetWriteArrayMetaData(False)
    writer.Write()
    print(time, name)

    file.write(f'{name}\t{time:.3f}\n')

  file.close()

  return 0

def create_single_sUbend_file(filename):
  mesh = pv.read(filename)
  # Save the VTK legacy file
  writer = vtk.vtkDataSetWriter()
  writer.SetFileTypeToASCII()
  writer.SetInputData(mesh)
  # Set the output file name with .vtk extension
  name = filename.replace('.vtu', '.vtk')
  writer.SetFileName(name)
  writer.SetFileVersion(42)
  writer.SetWriteArrayMetaData(False)
  writer.Write()
  return 0

## Create single sUbend file

In [5]:

# create_single_sUbend_file('sUbend.vtu')

0

In [15]:
# create_vti_file('FoV.vtu', 0.0005)
# create_vti_file('clipped.vtu', 0.0005)

In [14]:
# create_vti_file('sUbend.vtu', 0.0005)

In [24]:
# create_seed_file('FoV.vti', 'seeds.txt')
# create_seed_file('clipped.vti', 'entrance_seeds.txt')

In [3]:
data = np.genfromtxt('seeds.txt')
data

array([[ 0.00393563, -0.09035593, -0.01527433],
       [ 0.00443641, -0.09035593, -0.01527433],
       [ 0.00493718, -0.09035593, -0.01527433],
       ...,
       [ 0.08706461,  0.00326187,  0.01527344],
       [ 0.08756539,  0.00326187,  0.01527344],
       [ 0.08806617,  0.00326187,  0.01527344]])

In [4]:
data.shape

(1657140, 3)

In [13]:
sparse_data = data[np.random.choice(data.shape[0], 1000000, replace=True)]
sparse_data

array([[ 0.12462289, -0.04029294,  0.00125151],
       [ 0.10208792,  0.00025809,  0.0017523 ],
       [ 0.11911434, -0.0412942 , -0.00826336],
       ...,
       [ 0.00694029,  0.01527699,  0.00275386],
       [ 0.12312056, -0.05230806,  0.00625934],
       [ 0.122119  , -0.0863509 ,  0.00225308]])

In [14]:
np.savetxt('sparse_seeds.txt', sparse_data)